# Day 03

In [1]:
from numba import cuda
import numpy as np

## Process input data

In [2]:
with open("input.txt") as fh:
    inputs = fh.readlines()

In [22]:
slope = np.asarray([np.asarray([c == "#" for c in row.strip()]) for row in inputs])
slope

array([[False,  True, False, ..., False, False, False],
       [False,  True, False, ...,  True, False, False],
       [False, False, False, ..., False,  True, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False,  True,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

## Part 1

In [85]:
@cuda.jit
def calculate_trees(slope, answer):
    pos = cuda.grid(1)
    height, width = slope.shape
    if pos < height:
        if slope[pos][(pos * 3) % width]:
            cuda.atomic.add(answer, 0, 1)

In [86]:
threadsperblock = 128
blockspergrid = (len(slope) + (threadsperblock - 1)) // threadsperblock

In [87]:
answer = np.zeros(1)
calculate_trees[blockspergrid, threadsperblock](slope, answer)
answer

array([200.])

## Part 2

In [121]:
@cuda.jit
def calculate_many_trees(slope, answer):
    pos = cuda.grid(1)
    height, width = slope.shape
    if pos < height:
        
        if slope[pos][(pos * 1) % width]:
            cuda.atomic.add(answer, 0, 1)
            
        if slope[pos][(pos * 3) % width]:
            cuda.atomic.add(answer, 1, 1)
            
        if slope[pos][(pos * 5) % width]:
            cuda.atomic.add(answer, 2, 1)
            
        if slope[pos][(pos * 7) % width]:
            cuda.atomic.add(answer, 3, 1)
            
        if pos % 2 == 0:
            x = pos / 2
            if slope[pos][int(x) % width]:
                cuda.atomic.add(answer, 4, 1)


In [122]:
answer = np.zeros(5)
calculate_many_trees[blockspergrid, threadsperblock](slope, answer)
answer

array([ 66., 200.,  76.,  81.,  46.])

In [123]:
np.prod(answer)

3737923200.0